In [ ]:
#Install required packages

%pip install -U langgraph langchain_community langchain_openai langsmith langgraph-supervisor 

In [9]:
# Environment Variable Initialization

import getpass
import os

def _set_if_undefined(var_name: str):
    """
    Set an environment variable if it is not already defined.
    
    Args:
        var_name (str): Name of the environment variable to set.
    """
    if not os.environ.get(var_name):
        # Securely prompt the user for input without echoing it on screen
        os.environ[var_name] = getpass.getpass(f"Please provide your {var_name}: ")

# ---- Environment Variables Required ----

_set_if_undefined("OPENAI_API_KEY")         # API key for OpenAI models
_set_if_undefined("LANGSMITH_TRACING")      # Enable LangSmith tracing ("true" to enable)
_set_if_undefined("LANGSMITH_API_KEY")      # API key for LangSmith platform
_set_if_undefined("MODEL")                  # Model name (e.g., "gpt-4.1" "gpt-4o", "gpt-3.5-turbo")

In [ ]:
# Multi-Agent Orchestration with LangGraph:
#- Supervisor agent coordinates between specialized workers.
#- Workers: weather reporting agent, dressing planner agent, image generator.
# ---- Imports ----
import os
from langchain_openai import ChatOpenAI
from typing import Annotated
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_community.tools.openai_dalle_image_generation import (OpenAIDALLEImageGenerationTool)
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
# ---- LLM Setup ----
# Load the default model from environment variables
default_model = os.environ["MODEL"]
# Initialize the LLM (Large Language Model) interface
llm = ChatOpenAI(model=default_model)
api_wrapper = DallEAPIWrapper(model="dall-e-3")
dalle_tool = OpenAIDALLEImageGenerationTool(api_wrapper=api_wrapper)
# ---- Node Definitions ----
# Image generation Agent
image_generation_agent = create_react_agent(
    llm,
    tools=[dalle_tool],
    name="image_generation_agent",
    prompt=(
        "You are an image generator. Generate an image of a person dressed as suggusted in the given text. "
        "You may use tools."
    )
)
@tool
def weather_reporting_tool(city: Annotated[str, "name of the city"]):
    """Tool to fetch current weather data for a given city."""
    # (Simulated API call — replace with real API integration)
    json_data = {
        "weather": {
            "name": city,
            "main": "Rain",
            "description": "moderate rain",
            "temp": 284.2,
            "feels_like": 282.93,
            "temp_min": 283.06,
            "temp_max": 286.82,
            "pressure": 1021,
            "humidity": 60,
            "sea_level": 1021,
            "grnd_level": 910
        }
    }
    return json_data
# Weather Reporting Agent
weather_reporting_agent = create_react_agent(
    llm,
    tools=[weather_reporting_tool],
    name="weather_reporting_agent",
    prompt=(
        "You are a weather reporter. Report current weather for the provided city. "
        "You may use tools. Do not suggest what to wear."
    )
)
# Dressing Planner Agent
dressing_planner_agent = create_react_agent(
    llm,
    tools=[],
    name="dressing_planner_agent",
    prompt=(
        "You suggest dressing options based on the current weather. "
        "Prioritize 'feels like' temperature and consider wind conditions."
    )
)
# ---- Supervisor Setup ----
# System prompt guiding the supervisor's behavior
system_prompt = (
    "# Role and Objective"
    "You are a Supervisor Agent tasked with managing a conversation between multiple specialized workers: "
    "`weather_reporting_agent`, `dressing_planner_agent` and `image_generation_agent`."
    "Your goal is to find current weather for a given city, suggest dressing options based on the weather and"
    "generate a picture of a person dressing according to the suggestion."
    "orchestrate worker actions to resolve the user's request efficiently."
    "# Instructions"
    " - Persist through multiple steps until the task is fully complete."
    " - Always select the next worker based on context."
    " - Think step-by-step before choosing a worker and after receiving results."
    "# Reasoning Steps"
    "1. Analyze current state."
    "2. Plan the next best action."
    "3. Reflect after worker output."
    "4. Repeat until completion."
    "# Tool/Worker Use"
    "- `weather_reporting_agent`: gather or analyze weather."
    "- `dressing_planner_agent`: suggest clothing based on weather."
    "- `image_generation_agent`: generates image based on the given description."
)
# Create supervisor
builder = create_supervisor(
    agents=[weather_reporting_agent, dressing_planner_agent, image_generation_agent],
    model=llm,
    prompt=(system_prompt)
)

In [16]:
# Compile the graph
graph = builder.compile()

In [18]:
# ---- Simulation ----
for s in graph.stream(
    {"messages": [("user", "Karachi")]}, debug=True):
    print(s)
    print("============================")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'Karachi')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'Karachi')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='Karachi', additional_kwargs={}, response_metadata={}, id='f676f061-3c0a-49ea-81b7-f3ec25fd1020')]}
[1:tasks] Starting 1 task for step 1:
- supervisor -> {'is_last_step': False,
 'messages': [HumanMessage(content='Karachi', additional_kwargs={}, response_metadata={}, id='f676f061-3c0a-49ea-81b7-f3ec25fd1020')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [HumanMessage(content='Karachi', additional_kwargs={}, response_metadata={}, id='f676f061-3c0a-49ea-81b7-f3ec25fd1020'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qYFpfK6OCdgr32g4O4bZ8pbP', 'function': {'arguments': '{}', 'name': 'transfer_to_weather